In [1]:
# please enable if you are using a package manager like pip or conda
# Rajiv is using uv for package management hence these are commented out
# !pip install langchain_openai
# !pip install langchain_chroma
# !pip install langchain_groq
# !pip install langchain_community
# !pip install langchain_core
# !pip install langchain_ray
# !pip install chromadb


In [2]:
import pandas as pd
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_community.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.vectorstores import chroma
from langchain_community.llms import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
import ray
import os
import dask.dataframe as dd


In [3]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("CUDA device name:", torch.cuda.get_device_name(0))
    print("Current device index:", torch.cuda.current_device())


CUDA available: False
CUDA device count: 0


In [ ]:
# For MPS (Apple Silicon)
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print("MPS device not found.")


tensor([1.], device='mps:0')


In [ ]:
dataframe = pd.read_csv("filtered_data.csv")


In [ ]:
dataframe["Crop"] = dataframe["Crop"].replace("Paddy Dhan", "Paddy_Dhan")


In [ ]:
categories = dataframe['Crop'].unique()
# categories[2] = categories[2].replace("Paddy Dhan","Paddy_Dhan")
category_groups = {cat: dataframe[dataframe['Crop'] == cat] for cat in categories}


In [ ]:
!pip install langchain_huggingface


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct",model_kwargs={"device": "cuda"})


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_xlm-roberta_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [ ]:
embedding


HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large-instruct', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
dataframe = dataframe.rename(columns={"Season English": "Season_English"})


#### Renaming the calender names

In [ ]:
import calendar

month_number = 3
month_name = calendar.month_name[month_number]


In [ ]:
dataframe["Month"] = dataframe["Month"].apply(lambda x: calendar.month_name[x])


In [ ]:
dataframe[dataframe["Crop"]=="Tomato"].head(5)


,Unnamed: 0,BlockName,Category,Year,Month,Day,Crop,DistrictName,QueryType,Season,...,StateName,QueryText,KccAns,latitude,longitude,Disease,kcc_word_count,QueryText_word_count,Season_English,BlockNamenew
140468,568214,0,0,2009,May,3,Tomato,PULWAMA,29,JAYAD,...,JAMMU AND KASHMIR,farmer wants to control over aphids in tomato,spray with endosulfan 35ec 3ml in 1litre of w...,33.9490,75.0418,Aphids,9,8,Summer Cropping Season,NaN
140469,568215,0,0,2009,April,11,Tomato,BADGAM,29,KHARIF,...,JAMMU AND KASHMIR,how to control aphids in thge seedlings of tomato,spray melathion 50 ec 1 ml in 1 liter of water,34.0385,74.7360,Aphids,11,9,Monsoon Cropping Season,NaN
140470,568217,0,0,2009,June,8,Tomato,BADGAM,29,JAYAD,...,JAMMU AND KASHMIR,information about the control of aphids in tomato,spray with endosulfan 35 ec 2mllt of water,34.0385,74.7360,Aphids,8,8,Summer Cropping Season,NaN
140471,568218,0,0,2009,June,18,Tomato,BADGAM,29,JAYAD,...,JAMMU AND KASHMIR,information regarding control of aphids in tomato,spray malathion 35ec 25mllitre of water,34.0385,74.7360,Aphids,6,7,Summer Cropping Season,NaN
140472,568220,0,0,2009,January,20,Tomato,JAMMU,29,KHARIF,...,JAMMU AND KASHMIR,control of aphids in tomato crop,spray metasystox 25ec 2-3mllt of water,32.7186,74.8581,Aphids,6,6,Monsoon Cropping Season,NaN


In [ ]:
category_groups = {cat: dataframe[dataframe['Crop'] == cat] for cat in categories}


In [ ]:
import chromadb

# chromadb.api.client.SharedSystemClient.clear_system_cache()


In [ ]:
from tqdm import tqdm


In [ ]:
# Store documents per category
for cat, df_cat in tqdm(category_groups.items(),desc="Processing categories"):

    print(len(df_cat))
    docs = [
        Document(page_content=f"DistrictName:{row['DistrictName']}\nStateName:{row['StateName']}\nSeason_English:{row['Season_English']}\nMonth:{row['Month']}\nDisease:{row['Disease']}\nQueryText:{row['QueryText']}\nKccAns:{row['KccAns']}", metadata={"category": cat})
        for _, row in df_cat.iterrows()
    ]

    chroma_collection = Chroma.from_documents(
        docs,
        embedding=embedding,
        persist_directory="./chroma_capstone_db_new",
        collection_name=cat
    )

    # chroma_collection.persist()


Processing categories:   0%|          | 0/5 [00:00<?, ?it/s]

5665


Processing categories:  20%|██        | 1/5 [00:29<01:57, 29.43s/it]

7453


Processing categories:  40%|████      | 2/5 [01:06<01:41, 33.89s/it]

...

Processing categories:  60%|██████    | 3/5 [09:33<08:20, 250.01s/it]

15741


Processing categories:  80%|████████  | 4/5 [10:44<02:59, 179.24s/it]

13613


Processing categories: 100%|██████████| 5/5 [11:43<00:00, 140.77s/it]


In [ ]:
chroma_db = Chroma(
    persist_directory="./chroma_capstone_db_new",
    embedding_function=embedding,
    collection_name="Tomato"  # Specify which collection to load
)


In [ ]:
print(chroma_db._collection.count())


13613


In [ ]:
question = "give me the cure for tomato plant"
docs = chroma_db.max_marginal_relevance_search(question, k=3, fetch_k=6)


In [ ]:
retriever = chroma_db.as_retriever(search_type="mmr", search_kwargs={"k": 3, "fetch_k":6})
docs = retriever.invoke(question)


In [ ]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser


In [ ]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""

QA_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)


In [ ]:
retrieval = RunnableParallel(
    {
        "context": RunnablePassthrough(context= lambda x: x["question"] | retriever),
        "question": RunnablePassthrough()
        }
    )


In [ ]:
chat_llm =  (
    repo_id="Qwen/Qwen3-4B-Thinking-2507",
    task="text-generation",
    max_new_tokens = 100,
    top_k = 30,
    temperature = 0.5,
    repetition_penalty = 1.03,
)


In [ ]:
#from langchain_community.chat_models import ChatHuggingFace
from langchain_huggingface import ChatHuggingFace
from langchain.chains import RetrievalQA


In [ ]:
llm = ChatHuggingFace(llm=chat_llm)


In [ ]:
question = "give me the cure for tomato plant in JAMMU AND KASHMIR for Aphids"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=chroma_db.as_retriever(), return_source_documents=True)

result = qa_chain.invoke({"query": question})


In [ ]:
result


{'query': 'give me the cure for tomato plant in JAMMU AND KASHMIR for Aphids',
 'result': 'Let me analyze the information provided in the context to answer the question about treating aphids in tomato plants in JAMMU AND KASHMIR.\n\nI need to look for entries where:\n1. DistrictName: JAMMU AND KASHMIR (or more specifically JAMMU, as JAMMU is a part of JAMMU AND KASHMIR)\n2. Disease: Aphids\n3. QueryText related to aphids in tomato\n\nHere are the relevant entries from the context:\n\n1. DistrictName: DODA\n   StateName: JAMMU AND KASHMIR\n   Month: May\n   Disease: Aphids\n   QueryText: aphids in tomato crop\n   KccAns: spray chloroperiphous 2ml in 1 lt water\n\n2. DistrictName: JAMMU\n   StateName: JAMMU AND KASHMIR\n   Month: March\n   Disease: Aphids\n   QueryText: farmer wants information about the control of aphids in tomato\n   KccAns: spray malathion 2 ml lt of water\n\n3. DistrictName: PULWAMA\n   StateName: JAMMU AND KASHMIR\n   Month: May\n   Disease: Aphids\n   QueryText: ap

In [ ]:
import shutil

# Zip the Chroma DB folder
shutil.make_archive("chroma_capstone_db_new", 'zip', "./chroma_capstone_db_new")


'/content/chroma_capstone_db_new.zip'

In [ ]:
from google.colab import files
files.download("chroma_capstone_db_new.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
category_groups.keys()


dict_keys(['Apple', 'Coconut', 'Paddy_Dhan', 'Potato', 'Tomato'])